# Esercizio LU
### Numero di condizionamento
Si calcola il numero di condizionamento della matrice con le funzioni di libreria.
### Risoluzione con fattorizzazione LU con pivoting
Tramite le funzioni di libreria si calcola il risultato del problema test. La funzione scipy.linalg.lu_factor implementa la fattorizzazione LU con pivoting. La funzione restituisce una matrice LU che avrà nella parte triangolare supreiore gli elementi della matrice U e nella parte triangolare inferiore gli elementi di L; inoltre viene restituita la matrice di permutazione P. Infine la funzione di libreria scipy.linalg.lu_solve risolve il sistema a partire dalla matrice LU, dalla matrice P e dal termine noto b.
### Errore in norma 2
Una volta risolto il problema si calcola l'errore un norma 2 con la formula:
$$ \frac {\|x^*-x\|} {\|x\|} $$

In [ ]:
import numpy
import scipy

def fattorizzazioneLU(A,b,xTrue):
    cond=numpy.linalg.cond(A)
    LU, P=scipy.linalg.lu_factor(A)
    x=scipy.linalg.lu_solve((LU,P), b)
    errNorma2=scipy.linalg.norm(x-xTrue)/scipy.linalg.norm(xTrue)
    return (cond,x,errNorma2)

### Condizionamento all'aumentare della dimensione
Iterando sulla dimensione della matrice e tenendo traccia del condizionamento si crea un grafico che ogni volta è diverso e senza un andamento individuabile in quanto la dimensione della matrice non influisce sul suo condizionamento.
### Errore all'aumentare della dimensione
Si nota invece che la l'errore ha una relazione con il condizionamento della matrice: si hanno i picchi negli stessi punti in entrambi i frafi.

![grafico](./fattLU.png "Grafico fattorizzazione LU")

# Esercizio Cholesky
### Numero di condizione
Si calcola il numero di condizione delle matrici dei problemi test e si salvano in un vettore.
### Errore in norma 2
Si calcola l'errore in norma 2 dei problemi test. La formula per farlo è:
$$ \frac {\|x^*-x\|} {\|x^*\|} $$
### Risoluzione con fattorizzazione di Cholesky
Formula risolutiva della fattorizzazione di Cholesky:
$$\begin{cases} Ly=b \\ L^T x=y \end{cases}$$
La fattorizzazione di Cholesky è svolta dalle funzioni di libreria: la funzione scipy.linalg.cholesky genera la matrice L che grazie al parametro lower=true è triangolare inferiore. Si risolve poi il sistema triangolare inferiore $Ly=b$ per poi generare la trasposta di $L$ e risolvere il sistema $L^T x=y$.


In [ ]:
import numpy
import scipy

def fattorizzazioneCholesky(A,b,xTrue):
    cond=numpy.linalg.cond(A)
    L=scipy.linalg.cholesky(A, lower=True)
    y=scipy.linalg.solve(L, b, lower=True)
    Lt=numpy.transpose(L)
    x=scipy.linalg.solve(Lt, y)
    errNorma2=scipy.linalg.norm(x-xTrue)/scipy.linalg.norm(xTrue)
    return (cond,x,errNorma2)

### Grafici
Si creano i grafici del condizionamento e dell'errore in norma 2.
#### Con matrice di Hilbert
Si nota che il malcondizionamento della matrice fa esplodere l'errore, è per questo che la dimensione si ferma a 14.

![grafico](./hilbert.png "Con matrice di Hilbert")
#### Con matrice Tridiagonale
Si nota che il condizionamento ha una crescita logaritmica che ha l'effetto di far diminuire l'errore sulla risoluzione.

![grafico](./tridiagCholesky.png "Con matrice tridiagonale")

# Esercizio Jacobi e Gauss-Seidel
### Numero di condizione
Si calcola il numero di condizione delle matrici dei problemi test e si salvano in un vettore.
### Errore in norma 2
Si calcola l'errore in norma 2 dei problemi test. La formula per farlo è:
$$ \frac {\|x^*-x\|} {\|x^*\|} $$
### Risoluzione con il metodo di Jacobi
Per il metodo di Jacoby si implementa la seguente formula elemento per elemento:
$$x_i^{(k)} = \frac { b_i - \sum_{j=1}^{i-1} {a_{ij} x_j^{(k-1)}} - \sum_{j=i+1}^n {a_{ij} x_j^{(k-1)}} } {a_{ii}}$$
Essa è implementata all'interno del while:
- il while itera tenendo conto del numero di iterazioni e uscendo quando si raggiunge maxit, valore passato come input della funzione.
- nel while si calcola l'errore in norma 2 e si controlla che rimanga al di sotto della tolleranza passata come input.
- il valore di $x_0$, che comunque sarebbe indifferente, viene preso come parametro della funzione.
- il valore di $x^{(k-1)}$ viene salvato nella variabile x_old  per essere impiegato nel calcolo di $x^{(k)}$.
- il for annidato nel while calcola il valore $x^{(k)}$ valore per valore con la formula indicata per questione di rapidità computazionale.

In [ ]:
import numpy as np
import scipy.linalg

def Jacobi(A,b,x0,maxit,tol, xTrue):
	n=np.size(x0)     
	ite=0
	x = np.copy(x0)
	norma_it=1+tol
	relErr=np.zeros((maxit, 1))
	errIter=np.zeros((maxit, 1))
	relErr[0]=np.linalg.norm(xTrue-x0)/np.linalg.norm(xTrue)
	while (ite<maxit-1 and norma_it>tol):
		x_old=np.copy(x)
		for i in range(0,n):
			x[i]=(b[i]-np.dot(A[i,0:i],x_old[0:i])-np.dot(A[i,i+1:n],x_old[i+1:n]))/A[i,i]
		ite=ite+1
		norma_it = np.linalg.norm(x_old-x)/np.linalg.norm(x_old)
		relErr[ite] = np.linalg.norm(xTrue-x)/np.linalg.norm(xTrue)
		errIter[ite-1] = norma_it
	relErr=relErr[:ite]
	errIter=errIter[:ite]  
	return [x, ite, relErr, errIter]

### Risoluzione con il metodo di Gauss-Seidel
Per il metodo di Gauss-Seidel si implementa la seguente formula elemento per elemento:
$$x_i^{(k)} = \frac { b_i - \sum_{j=1}^{i-1} {a_{ij} x_j^{(k)}} - \sum_{j=i+1}^n {a_{ij} x_j^{(k-1)}} } {a_{ii}}$$
Essa è implementata all'interno del while:
- il while itera tenendo conto del numero di iterazioni e uscendo quando si raggiunge maxit, valore passato come input della funzione.
- nel while si calcola l'errore in norma 2 e si controlla che rimanga al di sotto della tolleranza passata come input.
- il valore di $x_0$, che comunque sarebbe indifferente, viene preso come parametro della funzione.
- il valore di $x^{(k-1)}$ viene salvato nella variabile x_old per essere impiegato nel calcolo di $x^{(k)}$.
- il for annidato nel while calcola il valore $x^{(k)}$ valore per valore con la formula indicata per questione di rapidità computazionale.

In [ ]:
import numpy as np
import scipy.linalg

def GaussSidel(A,b,x0,maxit,tol, xTrue):
	n=np.size(x0)
	ite=0
	x = np.copy(x0)
	norma_it=1+tol
	relErr=np.zeros((maxit, 1))
	errIter=np.zeros((maxit, 1))
	relErr[0]=np.linalg.norm(xTrue-x0)/np.linalg.norm(xTrue)
	while(ite<maxit-1 and norma_it>tol):
		x_old=np.copy(x)
		for i in range(0,n):
			x[i]=(b[i]-A[i,0:i]@x[0:i]-A[i,i+1:n]@x_old[i+1:n])/A[i,i]
		ite+=1
		norma_it = np.linalg.norm(x_old-x)/np.linalg.norm(x_old)
		relErr[ite] = np.linalg.norm(xTrue-x)/np.linalg.norm(xTrue)
		errIter[ite-1] = norma_it
	relErr=relErr[:ite]
	errIter=errIter[:ite]
	return [x, ite, relErr, errIter]

### Grafici
Si riporta nel grafico l'errore in norma 2 e il numero di iterazioni.
#### Dimensione fissata
Si nota che la convergenza del metodo di Gauss-Seidel è più rapida: la curva è più ripida nell'abbassarsi; e per questo motivo il medoto converge prima e la curva relativa al metodo di Gauss-Seidel si interrompe prima di quella di Jacobi.

![grafico](./errJacGSdimFissa.png "Con matrice tridiagonale")
#### Dimensione variabile
Si nota che nel complesso il numero di iterazioni e l'erroire relativo si appiattiscono e prendono un andamento logaritmico.

![grafico](./errJacGSdimVar.png "Con matrice tridiagonale")
#### Tempi di esecuzione
Si nota che i metodi iterativi aumentano il tempo di esecuzione al aumentare della dimensione, mentre quelli diretti hanno tempi di esecuzione minore.

![grafico](./tempi.png "Tempo di esecuzione")
## Traccia per la discussione
Spiegare:
- andamento dell’errore rispetto al numero di condizione della matrice.
    - Si nota bene nella matrice di Hilbert, che è molto mal condizionata, come, all'aumentare della sua dimensione, esplode anche il numero di iterazioni; tantè che non è possibile andare oltre una certa dimensione in quanto non è più possibile calcolare l'errore. Nel grafico della matriche tridiagomale con il metodo di Cholesky è invece evidente che a una crescita logaritmica del condizionamento equivale un'iperbole per quanto riguarda l'errore.
- l’andamento del tempo di esecuzione rispetto alla dimensione del sistema in relazione alla complessità computazioneale degli algoritmi utilizzati.
    - Si nota inanzitutto un lieve scarto tra il metodo diretto LU e quello di Cholesky, infatti la complessità computazionale del primo equivale a $\mathcal{O}(\frac{n^3}3)$, mentre quella del secondo è $\mathcal{O}(\frac{n^3}6)$. Cercare costo computazionale metodi iterativi...
- la differenza di errore e tempo di esecuzione ottenuti con i metodi diretti e iterativi:
    - La differenza di errore... . Mentre la differenza in termini di tempo tra metodi diretti e iterativi è data dal fatto che i primi sono esatti, quindi calcolano il risultato in un solo passaggio; mentre i secondi arrivano al valore ricercato tramite molti passaggi(le iterazioni) che appesantiscono l'esecuzione.